# Ollama를 활용한 오픈소스 모델 추론

## 학습 목표
- Ollama 설치 및 기본 명령어 사용
- 다양한 오픈소스 모델 비교 (LLaMA 3.2 vs EXAONE 3.5)
- Huggingface 모델을 Ollama에서 활용

## Ollama 소개

Ollama는 로컬 환경에서 대규모 언어 모델을 쉽게 실행할 수 있는 오픈소스 도구입니다.

### 주요 장점
- 간편한 설치와 사용
- 다양한 오픈소스 모델 지원
- 로컬 실행으로 데이터 프라이버시 보장
- REST API 제공
- 무료 사용 (토큰 제한 없음)

## 서버 실행 및 기본 명령어

In [ ]:
# 터미널 실행 명령어
# !ollama serve

In [ ]:
# Ollama 서버 시작
import subprocess, time


# 백그라운드에서 Ollama 서버 실행
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

time.sleep(3)
print("Ollama 서버가 시작되었습니다.")

In [ ]:
# 기본 명령어들
print("=== Ollama 버전 ===")
!ollama --version

print("\n=== 현재 실행 중인 모델 ===")
!ollama ps

print("\n=== 설치된 모델 목록 ===")
!ollama list

## LLaMA 3.2 모델 테스트

Meta에서 개발한 LLaMA 3.2 모델을 다운로드하고 테스트해보겠습니다.   
REST API를 통해 한국어 질문에 대한 응답을 확인하고, 영어 중심 모델의 한국어 성능 한계를 알아봅니다.

In [ ]:
# LLaMA 3.2 모델 다운로드
!ollama pull llama3.2:latest

In [ ]:
# LLaMA 3.2 한국어 성능 테스트
import requests, json


def generate_with_api(prompt, model="llama3.2:latest"):
    """Ollama API를 사용한 텍스트 생성"""
    url = "http://localhost:11434/api/generate"

    data = {"model": model, "prompt": prompt, "stream": False, "options": {"num_predict": 100, "temperature": 0.7}}
    try:
        response = requests.post(url, json=data, timeout=120)
        if response.status_code == 200:
            return json.loads(response.text)["response"]
        else:
            return f"API 오류: {response.status_code}"
    except Exception as e:
        return f"연결 오류: {str(e)}"


# LLaMA 3.2 한국어 테스트
print("=== LLaMA 3.2 한국어 성능 테스트 ===")
test_prompt = "안녕하세요. 한국어로 자기소개를 해주세요."
llama_response = generate_with_api(test_prompt, "llama3.2:latest")
print(f"질문: {test_prompt}")
print(f"LLaMA 3.2 응답: {llama_response}\n")

# ※ LLaMA 3.2는 한국어 성능이 제한적입니다. 더 나은 한국어 지원을 위해 EXAONE 3.5를 사용해보겠습니다

## EXAONE 3.5 모델 사용 (한국어 특화)

한국어에 특화된 EXAONE 3.5 2.4B 모델을 사용해보겠습니다.   
LLaMA 3.2와 비교하여 더 나은 한국어 응답 성능을 확인하고, 다양한 한국어 질문으로 테스트해봅니다.    

In [ ]:
# EXAONE 3.5 2.4B 모델 다운로드
!ollama pull exaone3.5:2.4b

In [ ]:
# EXAONE 3.5 한국어 성능 테스트
print("=== EXAONE 3.5 한국어 성능 테스트 ===")
exaone_response = generate_with_api(test_prompt, "exaone3.5:2.4b")
print(f"질문: {test_prompt}")
print(f"EXAONE 3.5 응답: {exaone_response}\n")

# 추가 한국어 테스트
korean_tests = ["머신러닝과 딥러닝의 차이점을 설명해주세요.", "파이썬 프로그래밍의 장점 3가지를 알려주세요."]

for i, prompt in enumerate(korean_tests, 1):
    print(f"=== 테스트 {i} ===")
    response = generate_with_api(prompt, "exaone3.5:2.4b")
    print(f"질문: {prompt}")
    print(f"응답: {response}\n")

## Huggingface 모델 활용

Huggingface에서 한국어 모델을 다운로드하고 GGUF 형식으로 변환하는 과정을 진행합니다.    
Kakao의 Kanana 모델을 FP16 형식으로 메모리를 절약하면서 로드하고,    
이후 GGUF 변환을 위해 저장합니다.    

In [ ]:
# Kakao Kanana 모델 다운로드
model_name = "kakaocorp/kanana-1.5-2.1b-instruct-2505"
output_dir = "./models/kakao-kanana"

!huggingface-cli download {model_name} --local-dir {output_dir}

### llama.cpp

Huggingface 모델을 GGUF 형식으로 변환하는 도구인 llama.cpp를 사용해보겠습니다.    
GitHub에서 llama.cpp를 클론하고 make 명령어로 빌드하여 변환 도구를 준비합니다.   
llama.cpp에서는 모델이 GGUF 파일 형식으로 저장되어 있어야 합니다.    
다른 데이터 형식의 모델도 convert_*.py를 사용해 GGUF로 변환할 수 있습니다.  

In [ ]:
# 아래 줄의 주석을 해제하면, 미리 받은 것이 아니라 최신 레포지토리를 직접 클론할 수 있습니다
# !rm -rf llama.cpp
# !git clone https://github.com/ggerganov/llama.cpp.git

import os

# 절대 경로로 모델 위치 확인
model_path = os.path.abspath("./models/kakao-kanana")
print(f"모델 절대 경로: {model_path}")

# 조건문으로 폴더가 없을 때만 클론하도록 함.
if not os.path.exists("llama.cpp"):
    !git clone https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp

### 모델을 INT8 모델로 변환

convert_hf_to_gguf.py 스크립트를 사용하여 Huggingface 모델을 GGUF 형식으로 변환합니다.    
q8_0 양자화를 적용하여 메모리 사용량을 줄이고,   
변환된 파일을 `“kakaocorp/kanana-1.5-2.1b-instruct-2505”`모델이 다운로드 된 경로에 저장합니다.   

In [ ]:
import os
import glob

# 현재 작업 디렉토리 확인
print(f"현재 디렉토리: {os.getcwd()}")

# 모델 경로 찾기
base_path = ""
if os.path.exists(base_path):
    possible_paths = glob.glob(f"{base_path}/*")
    if possible_paths:
        model_path = possible_paths[0]
        print(f"찾은 모델 경로: {model_path}")
    else:
        print("스냅샷 폴더를 찾을 수 없습니다.")
else:
    print(f"기본 경로가 존재하지 않습니다: {base_path}")

# 캐시 디렉토리 생성
!mkdir -p /home/elicer/.cache

# GGUF 변환 실행
!python convert_hf_to_gguf.py --outfile /home/elicer/.cache/kanana-q8_0.gguf --outtype q8_0 --verbose {model_path}

In [ ]:
# Modelfile 생성 (줄별로 작성)
modelfile_lines = [
    "FROM /home/elicer/.cache/kanana-q8_0.gguf",
    'TEMPLATE """{{ if .System }}<|im_start|>system',
    "{{ .System }}<|im_end|>",
    "{{ end }}{{ if .Prompt }}<|im_start|>user",
    "{{ .Prompt }}<|im_end|>",
    "{{ end }}<|im_start|>assistant",
    '"""',
    'PARAMETER stop "<|im_end|>"',
    'PARAMETER stop "<|im_start|>"',
    "PARAMETER temperature 0.7",
    "PARAMETER top_p 0.9",
    "PARAMETER top_k 40",
    "PARAMETER repeat_penalty 1.1",
]

# Modelfile 저장
with open("/home/elicer/Modelfile", "w") as f:
    for line in modelfile_lines:
        f.write(line + "\n")

print("Modelfile이 생성되었습니다.")

### 원본 FP16을 GGUF 포맷으로 변환

양자화를 적용하지 않고 원본 FP16 품질 그대로 GGUF 형식으로 변환하는 대안 방법입니다.    
현재는 주석 처리되어 있지만, 양자화 없이 원본 품질을 유지하고 싶을 때 사용할 수 있습니다.   

In [ ]:
#!python convert_hf_to_gguf.py --outtype f16 --verbose /home/elicer/.cache/huggingface/hub/models--MLP-KTLim--llama-3-Korean-Bllossom-8B/snapshots/ed9647c18477ee09a03690c613c859eddca24362

## Ollama에서 변환한 모델 실행하기

GGUF로 변환된 모델을 Ollama에 등록하고 실제로 추론을 실행해보겠습니다.    
Modelfile을 생성하여 정의한 모델 설정을 이용해, Ollama에서 커스텀 모델로 실행하는 방법을 학습합니다.    

### Ollama에 모델 추가하기

생성한 Modelfile을 사용하여 'kanana-q8'이라는 이름으로 커스텀 모델을 Ollama에 등록합니다.    

In [ ]:
!ollama create kanana -f /home/elicer/Modelfile

### 양자화된 모델 실행하기

등록된 kanana-q8 모델을 실행하여 실제 추론을 테스트해봅니다.    
"Tell me about yourself" 질문에 대한 응답을 생성하여 변환된 모델이 정상적으로 작동하는지 확인하고,     
응답 품질을 확인해봅니다.    

In [ ]:
!ollama run kanana "네가 누구인지 알려줘"

In [ ]:
generate_with_api("raspberry 안에 r은 몇 개 있니?", "kanana")